In [1]:
import pandas as pd
import numpy as np
import pm4py
from datetime import timedelta
from sklearn.tree import DecisionTreeRegressor

In [2]:
log = pm4py.read_xes("BPI Challenge 2017.xes") # reading in the xes file, make sure it is located in the directory

parsing log, completed traces :: 100%|██████████| 31509/31509 [01:25<00:00, 366.62it/s]


In [3]:
df = pm4py.convert_to_dataframe(log) # converting eventlog to dataframe
df

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,2017-01-16 09:51:21.139000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144


In [10]:
# prefix extraction

states = df["concept:name"].unique()
prefixes = []
for counter in range(len(states)):
    i = counter
    character = i % 26
    i //= 26
    j = "" + chr(character + ord('A'))
    while i != 0:
        character = i % 26
        i //= 26
        j = chr(character + ord('A')) + j
    prefixes.append(j)

sequences = dict(zip(states, prefixes))
sequences

{'A_Create Application': 'A',
 'A_Submitted': 'B',
 'W_Handle leads': 'C',
 'W_Complete application': 'D',
 'A_Concept': 'E',
 'A_Accepted': 'F',
 'O_Create Offer': 'G',
 'O_Created': 'H',
 'O_Sent (mail and online)': 'I',
 'W_Call after offers': 'J',
 'A_Complete': 'K',
 'W_Validate application': 'L',
 'A_Validating': 'M',
 'O_Returned': 'N',
 'W_Call incomplete files': 'O',
 'A_Incomplete': 'P',
 'O_Accepted': 'Q',
 'A_Pending': 'R',
 'A_Denied': 'S',
 'O_Refused': 'T',
 'O_Cancelled': 'U',
 'A_Cancelled': 'V',
 'O_Sent (online only)': 'W',
 'W_Assess potential fraud': 'X',
 'W_Personal Loan collection': 'Y',
 'W_Shortened completion ': 'Z'}

In [65]:
from pm4py.objects.log.util import interval_lifecycle
enriched_log = interval_lifecycle.assign_lead_cycle_time(log) # creating enriched log
pd.DataFrame(enriched_log[0]).set_index('concept:name')

,@@approx_bh_overall_wasted_time,@@approx_bh_partial_cycle_time,@@approx_bh_partial_lead_time,@@approx_bh_this_wasted_time,@@duration,@approx_bh_ratio_cycle_lead_time,Accepted,Action,CreditScore,EventID,...,OfferID,OfferedAmount,Selected,case:ApplicationType,case:LoanGoal,case:RequestedAmount,case:concept:name,org:resource,start_timestamp,time:timestamp
concept:name,,,,,,,,,,,,,,,,,,,,,
A_Create Application,0.000,0,0.000,0.000,0.0,1.0,NaN,Created,NaN,Application_652823628,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_1,2016-01-01 09:51:15.304000+00:00,2016-01-01 09:51:15.304000+00:00
A_Submitted,0.048,0,0.048,0.048,0.0,0.0,NaN,statechange,NaN,ApplState_1582051990,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_1,2016-01-01 09:51:15.352000+00:00,2016-01-01 09:51:15.352000+00:00
A_Concept,81.109,0,81.109,81.061,0.0,0.0,NaN,statechange,NaN,ApplState_642383566,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_1,2016-01-01 09:52:36.413000+00:00,2016-01-01 09:52:36.413000+00:00
A_Accepted,25724.696,0,25724.696,25643.587,0.0,0.0,NaN,statechange,NaN,ApplState_99568828,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_52,2016-01-02 11:23:04.299000+00:00,2016-01-02 11:23:04.299000+00:00
O_Create Offer,25724.696,0,25724.696,0.000,0.0,0.0,True,Created,979.0,Offer_148581083,...,NaN,20000.0,True,New credit,Existing loan takeover,20000.0,Application_652823628,User_52,2016-01-02 11:29:03.994000+00:00,2016-01-02 11:29:03.994000+00:00
O_Created,25724.696,0,25724.696,0.000,0.0,0.0,NaN,statechange,NaN,OfferState_1514834199,...,Offer_148581083,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_52,2016-01-02 11:29:05.354000+00:00,2016-01-02 11:29:05.354000+00:00
O_Sent (mail and online),25724.696,0,25724.696,0.000,0.0,0.0,NaN,statechange,NaN,OfferState_2051164740,...,Offer_148581083,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_52,2016-01-02 11:30:28.606000+00:00,2016-01-02 11:30:28.606000+00:00
A_Complete,25724.696,0,25724.696,0.000,0.0,0.0,NaN,statechange,NaN,ApplState_946455804,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_52,2016-01-02 11:30:28.633000+00:00,2016-01-02 11:30:28.633000+00:00
A_Validating,299980.669,0,299980.669,274255.973,0.0,0.0,NaN,statechange,NaN,ApplState_752879093,...,NaN,NaN,NaN,New credit,Existing loan takeover,20000.0,Application_652823628,User_117,2016-01-13 13:10:55.973000+00:00,2016-01-13 13:10:55.973000+00:00


In [66]:
# finding maximum number of traces

n = 10000
maximum = 0

for i in range(n): # maximum still returns 54 over the entire log, no need to iterate over entire dataset
    df_2 = pd.DataFrame(enriched_log[i]).set_index('concept:name')
    values = len(np.array(df_2['@@approx_bh_this_wasted_time']))
    if values > maximum:
        maximum = values

maximum

54

In [67]:
# finding number of unique applications

number_of_applications = len(df[df['EventID'].str.contains("Application")]['EventID'].unique())
number_of_applications

31509

In [68]:
complete_trace = np.zeros(maximum) # length of the longest trace
number_of_values = np.zeros(maximum) # length of the longest trace

#calculating average time per state, used to assign time predictions based on index

for i in range(number_of_applications):
    df_3 = pd.DataFrame(enriched_log[i]).set_index('concept:name')
    values = np.array(df_3['@@approx_bh_this_wasted_time'])
    for j in range(len(values)):
        complete_trace[j] += values[j]
        number_of_values[j] += 1
    

average_time_per_state = np.around(complete_trace/number_of_values, decimals = 2)
average_time_per_state

array([0.0000000e+00, 1.2000000e-01, 2.4892000e+03, 2.6187300e+04,
       5.2202500e+03, 5.0423000e+02, 2.9257000e+02, 1.7037970e+04,
       1.5800313e+05, 1.0673974e+05, 3.5046650e+04, 6.1275880e+04,
       6.0033520e+04, 2.8618230e+04, 2.9251130e+04, 3.2869860e+04,
       2.3110820e+04, 2.6997920e+04, 2.3705630e+04, 2.3600020e+04,
       1.9682000e+04, 1.6950760e+04, 1.8780960e+04, 2.0145630e+04,
       1.3891820e+04, 1.2900470e+04, 1.4434560e+04, 1.6539280e+04,
       1.2688560e+04, 1.5529020e+04, 1.4513750e+04, 6.4342000e+03,
       9.7649700e+03, 7.0125600e+03, 8.5711000e+03, 9.2357300e+03,
       2.3746820e+04, 8.3195900e+03, 7.4630000e+02, 8.4463800e+03,
       2.9247420e+04, 2.0883940e+04, 1.4105500e+03, 2.8288900e+03,
       6.1507800e+03, 7.5835300e+03, 2.2101000e+02, 5.9370500e+03,
       1.0000000e-02, 2.0000000e-02, 1.0000000e-02, 1.0000000e-02,
       1.0000000e-02, 1.0000000e-02])

In [69]:
#''' giving the most occuring state for each position '''

import numpy as np
from collections import Counter

n = 100 # this should be equal to the number of applications, but takes too long if it is set to number_of_applications (23 minutes)
# the workaround is replacing the last 17 elements with O_Cancelled; we tested this with the number of applications and that is what it added
counts = {}
most_common = []

for i in range(n): 
    values = np.array(pd.DataFrame(enriched_log[i]).set_index('concept:name').index)
    for j in range(len(values)):
        if j in counts:
            counts[j].append(values[j])
        else:
            counts[j] = [values[j]]

for counts in counts.values():
    most_common.append(max(counts, key = counts.count))

for i in range(17):
    most_common.append('O_Cancelled')

most_common


['A_Create Application',
 'A_Submitted',
 'A_Concept',
 'W_Complete application',
 'A_Accepted',
 'O_Create Offer',
 'O_Created',
 'O_Sent (mail and online)',
 'A_Complete',
 'O_Create Offer',
 'O_Created',
 'O_Sent (mail and online)',
 'A_Validating',
 'O_Cancelled',
 'O_Cancelled',
 'A_Validating',
 'A_Validating',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'A_Validating',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Create Offer',
 'O_Created',
 'O_Sent (online only)',
 'O_Returned',
 'A_Validating',
 'O_Accepted',
 'A_Pending',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled',
 'O_Cancelled']

In [71]:
#create dataframe with all unique applications to keep track of which state they are currently in
df_unique_apps = pd.DataFrame(df[df['EventID'].str.contains("Application")]['EventID'].unique()).assign(state_count = 0) 
df_unique_apps.columns = ['ApplicationID', 'state_count']

#create empty predicted next state & time columns for the original dataframe, to fill up later
df = df.assign(next_state = '', predicted_time = 0)
number_of_iterations = 1000 # approx. 0.14s per iteration. takes ~3 mins, scale down/up according to how long you want to wait
for i in range(number_of_iterations): 
    application_id = df['case:concept:name'][i] #get current application ID number in original dataframe
    df_unique_apps.loc[df_unique_apps['ApplicationID'].str.contains(application_id), ['state_count']] += 1 # increment app ID state by 1
    df_state = df_unique_apps.loc[df_unique_apps['ApplicationID'].str.contains(application_id), ['state_count']] 
    state_number = df_state.iloc[0]['state_count'] # converting state to integer (code above returns a dataframe)
    if state_number > 53:
        state_number = 53
    df.loc[i,'next_state'] = most_common[state_number] # assign next state at position i in dataframe the ith element in most_common
    df.loc[i,'predicted_time'] = df.loc[i,'time:timestamp'] + timedelta(seconds = average_time_per_state[state_number]) # assign time at position i in dataframe the ith element in average_time_per_state

predicted_data = df[0:number_of_iterations]
predicted_data.to_csv('predicted_data.csv', index = False)